<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/align_macaque_PBMC_data/2_virus_dlist_cdna/1_align_dlist_cdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align sequencing reads to PalmDB with kallisto translated search masking host transcriptomes using the D-list

In [ ]:
# Install kallisto from source
!git clone -q https://github.com/pachterlab/kallisto.git
!cd kallisto && mkdir build && cd build && cmake .. && make

# Install bustools from source
!git clone -q https://github.com/BUStools/bustools.git
!cd bustools && mkdir build && cd build && cmake .. && make

# Define paths to kallisto and bustools binaries
kallisto = "/content/kallisto/build/src/kallisto"
bustools = "/content/bustools/build/src/bustools"

In [ ]:
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

virus_fasta = "palmdb_rdrp_seqs.fa"
virus_t2g = "palmdb_clustered_t2g.txt"

In [ ]:
# Number of threads to use in alignment
threads = 8

### Download raw sequencing data

In [ ]:
!pip install -q ffq
import json

# Use ffq to get all FTP download links
out = "GSE158390_data.json"
!ffq GSE158390 --ftp -o $out

f = open(out)
data = json.load(f)
f.close()

print(len(data))

# # Download the complete dataset (106 paired fastqs containing a total of 30 billion reads)
# for dataset in data:
#     url = dataset["url"]
#     !curl -O $url

# Download only two paired fastqs to demonstrate this notebook
for dataset in data[:4]:
    url = dataset["url"]
    !curl -O $url

In [ ]:
import glob

fastqs = []
for filename in glob.glob("*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])

fastqs.sort()

# Get sample names
samples = []
for fastq in fastqs:
    samples.append(fastq.split("_")[0])

samples = list(set(samples))

# Generate sample batch file
sample_batch_file = "batch.txt"
with open(sample_batch_file, "w") as batch_file:
    for sample in samples:
        fastq1 = sample + "_1.fastq.gz"
        fastq2 = sample + "_2.fastq.gz"
        batch_file.write(sample + "\t" + fastq1 + "\t" + fastq2 + "\n")

### Align to PalmDB

In [ ]:
# Get host transcriptomes and concatenate them into a single file
!gget ref -w cdna -r 110 -d canis_lupus_familiaris
!gget ref -w cdna -r 110 -d macaca_mulatta
canine_cdna = "Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz"
macaque_cdna = "Macaca_mulatta.Mmul_10.cdna.all.fa.gz"

canine_macaque_fasta = "combined_transcriptomes.cdna.all.fa.gz"
!cat $canine_cdna $macaque_cdna > $canine_macaque_fasta

Generate virus index with masked host transcriptomes:

In [ ]:
virus_index = "virus_index.idx"
# Generate virus reference index
!$kallisto index \
    -t $threads \
    --aa \
    --d-list=$canine_macaque_fasta \
    -i $virus_index \
    $virus_fasta

Align to PalmDB and correct barcodes using host onlist:

In [ ]:
out_folder = "virus_dlist_cdna_alignment_results"

In [ ]:
%%time
!/usr/bin/time -v $kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t $threads \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0

In [ ]:
# Download cell barcode onlist generated during alignment to host
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/align_macaque_PBMC_data/bustools_onlist.txt

In [ ]:
%%time
!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w bustools_onlist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus